In [38]:
import numpy as np
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import dataset

prostate_whole_dataset = dataset.ProstatDS("dataset/prostate.txt")

In [40]:
test, val, train = dataset.dataset_split(prostate_whole_dataset, [0.2, 0.1])
train.normalize()
val.normalize(train.norm_values)
_ = test.normalize(train.norm_values)

# Gradient Check

In [41]:
from gradient_check import eval_numeric_gradient
from gradient_check import rel_error
import models

sample, _ = dataset.dataset_split(train, 40.0 / train.num_samples)
X = sample.X.astype("float64")

for reg in [0, 0.1]:
  model = models.TwoLayerNet(train.num_features, 20, reg=reg, dtype=np.float64)
  
  # Post-initialization: Make the 'b' params random values as well:
  for key in model.params.keys():
    p = model.params[key]
    if key.startswith('b'):
      model.params[key] = 0.01 * np.random.randn(*p.shape)

  print("Testing with reg=%.2e:" % reg)

  for key in model.params.keys():
    f = lambda : model.loss(X, sample.y)[0]
    grad_n = eval_numeric_gradient(f, model.params[key])
    grad_a = model.loss(sample.X, sample.y)[1][key]
    error = rel_error(grad_n, grad_a)
    print("%s, error=%.1e" % (key, error))


Testing with reg=0.00e+00:
W1, error=7.5e-07
b1, error=2.2e-06
W2, error=1.9e-07
b2, error=7.5e-08
Testing with reg=1.00e-01:
W1, error=4.9e-06
b1, error=4.7e-05
W2, error=5.2e-07
b2, error=8.4e-07


In [42]:
def accuracy(model):
  y_train = model.loss(train.X)
  acc_train = np.mean(y_train == train.y)
  y_val = model.loss(val.X)
  acc_val = np.mean(y_val == val.y)
  print("Accuracy: Training = %.1f%%, Validation = %.1f%%" % (acc_train * 100, acc_val * 100))

In [79]:
import models

# H = 12
reg = 1e-5
learning_rate = 3
learning_rate_decay = 0.99
# model = models.TwoLayerNet(train.num_features, H, reg=reg)
model = models.fc_multilayer_net([sample.num_features, 15, 2], reg=reg * 2)

# HACK: set the model2 parameters to refer to model parameters:
#model2.layers[0].W = model.params['W1']
#model2.layers[0].b = model.params['b1']
#model2.layers[2].W = model.params['W2']
#model2.layers[2].b = model.params['b2']

for i in range(300):
  loss, grad = model.loss(train.X, train.y)
  loss2, grad2 = model2.loss(train.X, train.y)

  for key in grad.keys():
    model.params[key] -= learning_rate * grad[key]
  if i % 10 == 0:
    learning_rate *= learning_rate_decay
  if i % 10 == 0:
    print("[%4d]: loss=%f" % (i, loss))
    accuracy(model)
  


[   0]: loss=0.858260
Accuracy: Training = 61.1%, Validation = 48.0%
[  10]: loss=0.546587
Accuracy: Training = 69.6%, Validation = 62.0%
[  20]: loss=0.568961
Accuracy: Training = 65.9%, Validation = 52.0%
[  30]: loss=0.545746
Accuracy: Training = 68.2%, Validation = 54.0%
[  40]: loss=0.526709
Accuracy: Training = 71.3%, Validation = 54.0%
[  50]: loss=0.518392
Accuracy: Training = 71.9%, Validation = 54.0%
[  60]: loss=0.478912
Accuracy: Training = 73.9%, Validation = 58.0%
[  70]: loss=0.452841
Accuracy: Training = 77.3%, Validation = 60.0%
[  80]: loss=0.418179
Accuracy: Training = 79.5%, Validation = 60.0%
[  90]: loss=0.383565
Accuracy: Training = 81.0%, Validation = 58.0%
[ 100]: loss=0.370580
Accuracy: Training = 81.5%, Validation = 58.0%
[ 110]: loss=0.386832
Accuracy: Training = 81.8%, Validation = 58.0%
[ 120]: loss=0.338641
Accuracy: Training = 82.1%, Validation = 60.0%
[ 130]: loss=0.342774
Accuracy: Training = 84.7%, Validation = 64.0%
[ 140]: loss=0.338646
Accuracy: Tr

In [80]:
y_pred = model.loss(test.X)
print("Test Accuracy: %.2f%%" % (np.mean(y_pred == test.y)))

Test Accuracy: 0.54%


#Trying a moduler Design

In [20]:
from models import fc_multilayer_net_cl

# Gradient-check the layers we've created:
sample, _ = dataset.dataset_split(train, 40.0 / train.num_samples)
X = sample.X.astype("float64")

H = 10
model = fc_multilayer_net_cl([sample.num_features, 10, 1], reg=0, dtype="float64")




[Affine(1)[15, 10]:float64, Relu, Affine(2)[10, 1]:float64]
